In [105]:
# !pip install opencv-python editdistance ml_collections wandb albumentations hwb
#!BEZIER_NO_EXTENSION=true pip install bezier

  Using cached hwb-0.0.15-py3-none-any.whl (8.0 kB)

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF

import torchvision as tv

import numpy as np

import typing as tp
import os

import cv2

import matplotlib.pyplot as plt
import pandas as pd
import math
import itertools

from editdistance import eval as edit_distance

from tqdm import tqdm

import errno

from ml_collections import ConfigDict

import importlib
import wandb

import albumentations as A

import inspect

In [79]:
from diploma_code.patched_hwb import HandWrittenBlot

# from diploma_code.model import (
#     ConvBlock, FusedInvertedBottleneck, ReduceBlock, Backbone, PositionalEncoding,
#     TransformerEncoder, CTCRawDecoder, CTCDecoderModel, ParallelModel, make_single_model, make_model
# )
from diploma_code.model_v2 import (
    Resnet34Backbone, BiLSTMEncoder, PositionalEncoding, TransformerEncoder, 
    CTCDecoderModel, ParallelModel, make_single_model_v2, make_model_v2
)
from diploma_code.transforms_torch import (
    VerticalRandomMasking, HorizontalResizeOnly, GaussianNoise, 
    RandomHorizontalStretch, RandomChoiceN
)
from diploma_code.char_encoder import (
    CharEncoder
)
# from diploma_code.data_loader.mjsynth import (
#     load_mjsynth_chars, load_mjsynth_samples
# )
from diploma_code.dataset import (
    BaseLTRDataset, LongLinesLTRDataset
)
from diploma_code.make_loader import (
    make_char_encoder, make_datasets, make_dataloaders
)
from diploma_code.optimizing import (
    pytorch_make_optimizer, StepLRWithWarmup, make_lr_scheduler
)
from diploma_code.utils import (
    log_metric_wandb, batch_to_device
)
from diploma_code.evaluation import (
    my_ctc_loss, my_dml_loss, decode_ocr_probs, get_edit_distance, 
    EpochValueProcessor, EpochDMLProcessor, CERProcessor
)
from diploma_code.trainer import (
    LTRTrainer
)

from diploma_code.config import default_diploma_config, get_config

from diploma_code.configs import BaseDatasetConfig
from diploma_code.configs import IamConfig



In [78]:
import diploma_code
import diploma_code.data_loader

importlib.reload(diploma_code.char_encoder)

importlib.reload(diploma_code.config)
importlib.reload(diploma_code.dataset)

importlib.reload(diploma_code.data_loader)
importlib.reload(diploma_code.evaluation)

importlib.reload(diploma_code.optimizing)

importlib.reload(diploma_code.model_v2)
importlib.reload(diploma_code.utils)
importlib.reload(diploma_code.trainer)

importlib.reload(diploma_code.make_loader)
importlib.reload(diploma_code.make_transforms)

importlib.reload(diploma_code.transforms_functional)

importlib.reload(diploma_code.configs)





<module 'diploma_code.configs' from '/Users/d.mokeev/MIPT/mutual_htr/diploma_code/configs/__init__.py'>

In [72]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [73]:
#!python3 ./train.py  --config=./diploma_code/config.py --config.wandb.run_name='sad_dogo'

In [80]:
cfg = get_config()
cfg.training.eval_epochs_interval = 1
cfg.training.eval_test_interval = 3
cfg.training.snapshot_epochs_interval = 3
cfg.data.root_path = './diploma_code/prepare_data/prepared_datasets/'
cfg.data.iam.path = './diploma_code/prepare_data/prepared_datasets/'

cfg.training.loader_num_workers = 1
cfg.evaluate.loader_num_workers = 1

cfg.device = 'cpu'

# cfg.training.load_from_checkpoint = True
# cfg.wandb.resume = True

In [84]:
import pickle

def pickle_trick(obj, max_depth=10):
    output = {}

    if max_depth <= 0:
        return output

    try:
        pickle.dumps(obj)
    except (pickle.PicklingError, TypeError) as e:
        failing_children = []

        if hasattr(obj, "__dict__"):
            for k, v in obj.__dict__.items():
                result = pickle_trick(v, max_depth=max_depth - 1)
                if result:
                    failing_children.append(result)

        output = {
            "fail": obj, 
            "err": e, 
            "depth": max_depth, 
            "failing_children": failing_children
        }

    return output

In [85]:
ltr = LTRTrainer(cfg)

In [86]:
ltr.train('cpu_looser')

  0%|▏                                                                                               | 2/1207 [00:34<5:49:14, 17.39s/it]


KeyboardInterrupt: 

In [22]:
ltr.train('sad_dogo')

  4%|██▊                                                                             | 114/3241 [00:07<03:30, 14.84it/s]


KeyboardInterrupt: 

In [246]:
path = "data/mjsynth/mnt/ramdisk/max/90kDICT32px"

bad_files = []

import time

count = 0
begin = time.time()

for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        if filename.split('.')[-1] == 'jpg':
            full_path = os.path.join(dirpath, filename)
            with open(full_path, 'rb') as im:
                try:
                    im.seek(-2,2)
                except OSError as e:
                    print(f'failed to process file {full_path}, error: {e}')
                    bad_files.append(full_path)
                    continue
                if im.read() != b'\xff\xd9':
                    print(f'{full_path} has premature end of jpeg file')
                    bad_files.append(full_path)
                    continue
            val = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
            if val is None:
                print(f'{full_path} has a corrupted image')
                bad_files.append(full_path)
            count += 1
            if count % 100000 == 0:
                print(f'Checked {count} images in {time.time() - begin} seconds, found {len(bad_files)} corrupted')
    

data/mjsynth/mnt/ramdisk/max/90kDICT32px/1696/4/211_Queened_61779.jpg has premature end of jpeg file
Checked 100000 images in 18.951529502868652 seconds, found 1 corrupted
data/mjsynth/mnt/ramdisk/max/90kDICT32px/2540/4/246_SQUAMOUS_73902.jpg has premature end of jpeg file
Checked 200000 images in 37.68372654914856 seconds, found 2 corrupted
Checked 300000 images in 56.298471212387085 seconds, found 2 corrupted
Checked 400000 images in 75.03809976577759 seconds, found 2 corrupted
Checked 500000 images in 93.85237288475037 seconds, found 2 corrupted
data/mjsynth/mnt/ramdisk/max/90kDICT32px/2489/4/221_snored_72290.jpg has premature end of jpeg file
Checked 600000 images in 112.51990985870361 seconds, found 3 corrupted
Checked 700000 images in 131.21052145957947 seconds, found 3 corrupted
Checked 800000 images in 150.05990934371948 seconds, found 3 corrupted
Checked 900000 images in 168.71192383766174 seconds, found 3 corrupted
Checked 1000000 images in 187.46954560279846 seconds, found 3

Checked 7500000 images in 1446.9203684329987 seconds, found 27 corrupted
failed to process file data/mjsynth/mnt/ramdisk/max/90kDICT32px/913/4/231_randoms_62372.jpg, error: [Errno 22] Invalid argument
Checked 7600000 images in 1466.345933675766 seconds, found 28 corrupted
Checked 7700000 images in 1485.691297531128 seconds, found 28 corrupted
Checked 7800000 images in 1504.955777645111 seconds, found 28 corrupted
Checked 7900000 images in 1524.2437522411346 seconds, found 28 corrupted
data/mjsynth/mnt/ramdisk/max/90kDICT32px/384/4/220_bolts_8596.jpg has premature end of jpeg file
failed to process file data/mjsynth/mnt/ramdisk/max/90kDICT32px/1817/2/363_actuating_904.jpg, error: [Errno 22] Invalid argument
Checked 8000000 images in 1543.4584033489227 seconds, found 30 corrupted
Checked 8100000 images in 1562.727600812912 seconds, found 30 corrupted
data/mjsynth/mnt/ramdisk/max/90kDICT32px/905/4/234_Postscripts_59142.jpg has premature end of jpeg file
Checked 8200000 images in 1582.0282